In [106]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np
from sojo.pstabilizer import group_instructorss_by_qubits, Instructor
from sojo.pstabilizer import weightss_to_lambda, instructor_to_lut
from sojo.utils import word_to_index, char_to_index, index_to_word, create_word_zj
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
def map_noncx(indices: np.ndarray, lambdas: np.ndarray, num_qubits: int, k: int):
	"""Given a list of indices and lambdas, return the indices and lambdas of the non-cx gates.
	Example:
		Initial: 1*IZ => indices = [3], lambdas = [1]
		Another: 2*XZ + 3*YY => indices = [7, 10], lambdas = [2, 3]
	Args:
		indices (np.ndarray)
		lambdas (np.ndarray)
		num_qubits (int)
		k: index of the operator
	Returns:
		np.ndarray, np.ndarray: mapped indices and lambdas
	"""
	weightss = []
	for index in indices:
		weights = []
		word = index_to_word(index, num_qubits)
		for j, char in enumerate(word):
			i_in_lut = char_to_index(char) - 1
			if i_in_lut == -1:
				weights.append([1,0,0,0])
			else: 
				weights.append(LUT[k][j][i_in_lut])
		weightss.append(weights)
	# Weightss's now a 4-d tensor
	weightss = np.array(weightss)
	# Flattening the weightss into new lambdas, => Can be process effeciently on hardware
	lambdas = weightss_to_lambda(weightss, lambdas)
	# For most of the cases, lambdas will be sparse
	# In the worst case, it will be full dense,
	# and the below lines would be useless.
	indices = np.nonzero(lambdas)[0]
	lambdas = np.array(lambdas)[indices]
	return indices, lambdas

def map_cx(indices: np.ndarray, lambdas: np.ndarray, cxs: np.ndarray, num_qubits: int):
	"""Mapping multple CX gates on a given indices and lambdas

	Args:
		indices (np.ndarray): Words after converting to integers
		lambdas (np.ndarray): Corresponding lambdas
		cxs (np.ndarray): List of [control, target] pairs from k^th CX-operator
		num_qubits (int)

	Returns:
		indices, lambdas
	"""
	for control, target in cxs:
		df = pl.read_csv(f'./sojo/db/{num_qubits}_{control}_{target}_cx.csv')
		selected_rows = df[indices]['out'].to_numpy()
		indices = np.abs(selected_rows)
		lambdas[np.where(selected_rows < 0)[0]] *= -1
	return indices, lambdas

In [159]:
ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.operatoring()

In [157]:
num_qubits = 3
ins = Instructor(num_qubits)
for k in range(5):
	for i in range(num_qubits - 1):
		ins.append('cx', [i, i + 1])
	ins.append('cx', [num_qubits - 1, 0])
	for i in range(num_qubits):
		ins.append('rx', i, np.random.rand())
		ins.append('ry', i, np.random.rand())
		ins.append('rz', i, np.random.rand())

ins.operatoring()

In [160]:
ins.orders

[0, 1, 0]

In [161]:
LUT = instructor_to_lut(ins)
LUT[0]

(2, 4, 3, 4)


array([[[ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.71091354,  0.70327942],
        [ 0.        ,  0.        , -0.70327942,  0.71091354]],

       [[ 0.        ,  0.        ,  0.        ,  1.        ],
        [ 0.        ,  0.        , -1.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  1.        ],
        [ 0.        ,  0.        , -1.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ]]])

In [ ]:

num_qubits = 4
lambdas = np.array([1])
indices = np.array([word_to_index(create_word_zj(0, num_qubits))])
print(ins.is_cx_first)
for order in ins.orders:
	if order == 0:
		indices, lambdas = map_cx(indices, lambdas, ins.xoperators[k_index], num_qubits)
	else:
     	indices, lambdas = map_noncx(indices, lambdas, num_qubits, k_index)


False


IndexError: list index out of range

In [149]:


# Example usage:
n = 5
m = 4
is_0_first = True
print(create_alternating_array(n, m, is_0_first))  # Output: [0, 1, 0, 1, 0, 1, 0, 1]


[0, 1, 0, 1, 0, 1, 0, 1, 0]


In [102]:
print(indices)
print(lambdas)

[4, 13]
[1, 1]


In [103]:
import polars as pl



map_cx(np.array([3, 1, 27, 5]), np.array([1,2,3,4]), [[0, 2], [1, 0]], 3)

(array([63,  1, 54, 21], dtype=int64), array([1, 2, 3, 4]))